In [2]:
import pandas as pd

In [3]:
# create csv file
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Renaming School column "name"
school_data.rename({"name" : "school_name"}, axis=1, inplace=True)


# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


In [4]:
#calculate total # of schools
total_schools = len(school_data_complete['school_name'].unique())

#calculate total # of students
total_students = school_data_complete['Student ID'].count()


#calculate total budget
total_budget = school_data['budget'].sum()

In [5]:
# Calculate the average math score
avg_math_score = school_data_complete['math_score'].mean()

#Calculate the average reading score
avg_reading_score = school_data_complete['reading_score'].mean()



In [6]:
#Calculate the percentage of students with a passing math score (70 or greater)
pass_math = len(school_data_complete.loc[school_data_complete['math_score'] >= 70])
per_math = (pass_math)/ total_students * 100


In [7]:
#Calculate the percentage of students with a passing reading score (70 or greater)
pass_reading = len(school_data_complete. loc[school_data_complete['reading_score'] >= 70])
per_reading = (pass_reading)/ total_students * 100

In [8]:
pass_mathreading = (school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score']>=70)].count()["student_name"])


In [9]:
#Calculate the percentage of students who passed math and reading (% Overall Passing)
overall = (pass_mathreading)/total_students * 100

In [10]:
# Create dataframe to hold the above results for district summary
district_summary = pd. DataFrame({"Total # of Schools": total_schools,
                                "Total # of Students": total_students,
                                "Total Budget": total_budget,
                                "Average Math Score": avg_math_score,
                                "Average Reading Score": avg_reading_score,
                                "% Passing Math": per_math,
                                "% Passing Reading": per_reading,
                                "% Overall Passing Rate": [overall]})

In [11]:
#Reorganize columns with double brackets
district_summary = district_summary[["Total # of Schools", "Total # of Students", "Total Budget", 
                                     "Average Math Score", "Average Reading Score", 
                                     "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

# Format to "$0,000.00"
district_summary['Total Budget'] = district_summary['Total Budget']. map('${:,.2f}'. format)

district_summary


,Total # of Schools,Total # of Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [12]:
renamed_student_df = student_data.rename(columns={"school" : "school name"})


In [13]:
# School Summary 
# We will delete the School ID column because we will not need it
del school_data_complete['School ID']


In [14]:
school_data_complete['Per Student Budget'] = school_data_complete['budget']/school_data_complete['size']

In [48]:
## Average Math Score & Average Reading Score
## Use a groupby function to group on school name and display both reading and math scores from above 
## then reset the index 
avg_math_read_tbl = student_data.groupby(['school_name'])['reading_score', 'math_score'].mean().reset_index() # avg_math_read_


<ipython-input-48-3017f26f6992>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_math_read_tbl = student_data.groupby(['school_name'])['reading_score', 'math_score'].mean().reset_index() # avg_math_read_


In [49]:
school_data_complete = school_data_complete.merge(avg_math_read_tbl, on= 'school_name', how = 'outer')

In [50]:
#Find the % of people who passed reading
summary_passing_read = renamed_student_df[renamed_student_df['reading_score']>=70]

#Find the % of people who passed math
summary_passing_math = renamed_student_df[renamed_student_df['math_score']>=70]

In [51]:
pass_read_count_sum = summary_passing_read.groupby(['school_name'])['reading_score'].count().reset_index()

## Then, rename the column the 'reading_score' to 'Reading Count'
pass_read_count_sum.rename({'reading_score' : 'Reading Count'}, axis=1, inplace=True)


In [52]:
# Count the number of students passing in math
pass_math_count_sum = summary_passing_math.groupby(["school_name"])['math_score'].count().reset_index()

# Renaming column from math_score to math count
pass_math_count_sum.rename({'math_score':'Math Count'}, axis=1,inplace=True)

In [53]:
#Merge pass read count with pass read count
pass_count = pass_math_count_sum.merge(pass_read_count_sum, on= "school_name", how='inner')

In [54]:
## Merge copy_school_sum with pass_count we just created
## We want to merge on School Name and by 'outer' to include everything

school_data_complete = school_data_complete.merge(pass_count, on="school_name", how='outer')

#copy_school_sum

In [55]:
## Calc % passing math and reading
## Take the subject count and divide by the school size, then pmultiply by 100 to get percentage

# % Passing Math
school_data_complete['% Passing Math'] = (school_data_complete['Math Count']/school_data_complete['size'])*100

# % Passing Reading
school_data_complete['% Passing Reading'] = (school_data_complete['Reading Count']/school_data_complete['size'])*100

#copy_school_sum

In [56]:
# Deleting the Math Count and Reading Count cols from copy_school_sum 
#Will not need the counts for future use

del school_data_complete['Math Count']
del school_data_complete['Reading Count']

#copy_school_sum

In [57]:
## Calc % overall passing 
## Overall Passing Rate (Average of the above two)

school_data_complete['% Overall Passing'] = (school_data_complete['% Passing Math'] + school_data_complete['% Passing Reading'])/2

#copy_school_sum

In [58]:
# now, rename axis for reading and math scores to Avg. Reading Score and Avg. Math Score in copy_school_sum
school_data_complete.rename({'reading_score':'Avg. Reading Score',
                             'math_score':'Avg. Math Score'}, axis=1, inplace=True)


In [59]:
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score_x,math_score_x,type,size,budget,Per Student Budget,reading_score_y,math_score_y,% Passing Math,% Passing Reading,% Overall Passing,Spending Level,Avg. Reading Score,Avg. Math Score
0,0,Paul Bradley,M,9th,Huang High School,66,79,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171,$646-675,81.182722,76.629414
1,1,Victor Smith,M,12th,Huang High School,94,61,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171,$646-675,81.182722,76.629414
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171,$646-675,81.182722,76.629414
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171,$646-675,81.182722,76.629414
4,4,Bonnie Ray,F,9th,Huang High School,97,84,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,73.500171,$646-675,81.182722,76.629414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,95.290520,$616-645,83.848930,83.418349
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,95.290520,$616-645,83.848930,83.418349
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,95.290520,$616-645,83.848930,83.418349
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,95.290520,$616-645,83.848930,83.418349


In [60]:
## Creating table that highlights the top 5 performing schools based on Overall Passing Rate. 
## Found the top 5 performing by sorting copy_school_sum on the '% Overall Passing' col in copy_school_sum df 
## By sorting, we can find the five top performing

top_performing_by_pr_df = school_data_complete.sort_values(by=['% Overall Passing'], ascending=False).head(5)

In [61]:
top_performing_by_pr_df.head(5)


,Student ID,student_name,gender,grade,school_name,reading_score_x,math_score_x,type,size,budget,Per Student Budget,reading_score_y,math_score_y,% Passing Math,% Passing Reading,% Overall Passing,Spending Level,Avg. Reading Score,Avg. Math Score
17168,17168,James Lopez,M,9th,Cabrera High School,99,89,Charter,1858,1081356,582.0,83.97578,83.061895,94.133477,97.039828,95.586652,$0-585,83.97578,83.061895
17247,17247,Steven Murphy,M,11th,Cabrera High School,96,76,Charter,1858,1081356,582.0,83.97578,83.061895,94.133477,97.039828,95.586652,$0-585,83.97578,83.061895
17259,17259,Alexandra Ramirez,F,10th,Cabrera High School,93,93,Charter,1858,1081356,582.0,83.97578,83.061895,94.133477,97.039828,95.586652,$0-585,83.97578,83.061895
17258,17258,Mr. Christopher Cook,M,12th,Cabrera High School,92,72,Charter,1858,1081356,582.0,83.97578,83.061895,94.133477,97.039828,95.586652,$0-585,83.97578,83.061895
17257,17257,Sherry Rivera,F,12th,Cabrera High School,76,70,Charter,1858,1081356,582.0,83.97578,83.061895,94.133477,97.039828,95.586652,$0-585,83.97578,83.061895


In [62]:
## Create a table that highlights the 5 worst performing schools based on Overall Passing Rate. 
## Include all of the same metrics as above.
## Using the copy_school_sum df found the bottom 5 performing schools based on Overall Passing Rate col
worst_performing_by_pr_df = school_data_complete.sort_values(by=['% Overall Passing']).head(5)

In [63]:
worst_performing_by_pr_df


,Student ID,student_name,gender,grade,school_name,reading_score_x,math_score_x,type,size,budget,Per Student Budget,reading_score_y,math_score_y,% Passing Math,% Passing Reading,% Overall Passing,Spending Level,Avg. Reading Score,Avg. Math Score
27091,27091,Bruce Brown,M,10th,Rodriguez High School,65,69,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,73.293323,$616-645,80.744686,76.842711
27377,27377,Emily Johnson,F,12th,Rodriguez High School,84,72,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,73.293323,$616-645,80.744686,76.842711
27376,27376,Robert Jensen,M,11th,Rodriguez High School,91,82,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,73.293323,$616-645,80.744686,76.842711
27375,27375,Robert Wang,M,10th,Rodriguez High School,76,75,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,73.293323,$616-645,80.744686,76.842711
27374,27374,Diana Flores,F,12th,Rodriguez High School,89,84,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,73.293323,$616-645,80.744686,76.842711


In [64]:
# We will use the pivot table group to display the requested information
math_scores_by_grade_df = pd.pivot_table(student_data, values=['math_score'], index=['school_name'], 
                                         columns=['grade'])
math_scores_by_grade_df = math_scores_by_grade_df.reindex(labels=['9th',
                                                                    '10th',
                                                                    '11th',
                                                                    '12th'], axis=1, level=1)

In [65]:
math_scores_by_grade_df


math_score                                 
grade                        9th       10th       11th       12th
school_name                                                      
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

In [66]:
# will use the pivot table group to display
read_scores_by_grade_df = pd.pivot_table(student_data, values=['reading_score'], index=['school_name'], 
                                         columns=['grade'])
read_scores_by_grade_df = read_scores_by_grade_df.reindex(labels=['9th',
                                                                    '10th',
                                                                    '11th',
                                                                    '12th'], axis=1, level=1)


In [67]:
read_scores_by_grade_df


reading_score                                 
grade                           9th       10th       11th       12th
school_name                                                         
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [68]:
# Copy the copy_school_sum and save as scores_by_school_spending
scores_by_school_spending = school_data_complete.copy()

# Print columns of each to verify they're the same
# copy_school_sum
# scores_by_school_spending.columns

In [69]:
# Create bins - we will need labels and bins
bins = [0, 585, 615, 645, 675]
spending_labels = ['$0-585', '$586-615', '$616-645', '$646-675']

In [70]:
# Use bins and labels to sort through data and divide it up appropriately 
# save bined data as bins_school_spending variable 
bins_school_spending = pd.cut(scores_by_school_spending['Per Student Budget'], bins, labels=spending_labels)

# Convert bins_school_spending to df
bins_school_spending = pd.DataFrame(bins_school_spending)

# add Spending Level col 
school_data_complete['Spending Level'] = bins_school_spending

In [71]:
# Show cols for bins_school_spending to verify 
#bins_school_spending.columns

In [72]:
# Do a groupby on Spending Level and school name
scores_by_school_spending = school_data_complete.groupby(['Spending Level'])['Avg. Reading Score',
                                                         'Avg. Math Score',
                                                         '% Passing Reading',
                                                         '% Passing Math',
                                                         '% Overall Passing'
                                                         ].mean()

<ipython-input-72-dc502c3f48cb>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  scores_by_school_spending = school_data_complete.groupby(['Spending Level'])['Avg. Reading Score',


In [73]:
scores_by_school_spending

,Avg. Reading Score,Avg. Math Score,% Passing Reading,% Passing Math,% Overall Passing
Spending Level,,,,,
$0-585,83.964039,83.363065,96.686558,93.702889,95.194724
$586-615,83.838414,83.529196,95.886889,94.124128,95.005509
$616-645,81.434088,78.061635,83.614770,71.400428,77.507599
$646-675,81.005604,77.049297,81.109397,66.230813,73.670105


In [74]:

# Create a copy of copy_school_sum and save as scores_by_school_size
scores_by_school_size = school_data_complete.copy()

# Print scores_by_school_size to verify
# scores_by_school_size

In [75]:
# Create bins to organize by size
bins = [0, 1000, 2000, 5000]
size_labels = ['Small', 'Medium', 'Large']


In [76]:
# Using bins and labels to sort through data and divide it up correctly 
# save bin data as bins_school_size variable 
bins_school_size = pd.cut(scores_by_school_size['size'], bins, labels = size_labels)

# Convert bins_school_spending to df
bins_school_size = pd.DataFrame(bins_school_size)

# add 'School Population' col 
school_data_complete['School Population'] = bins_school_size

In [77]:
# Show cols for bins_school_size to verify 
# bins_school_size.columns

In [78]:
# Do a groupby on School Population and school name
scores_by_school_size = school_data_complete.groupby(['School Population','school_name'])['Avg. Reading Score',
                                                         'Avg. Math Score',
                                                         '% Passing Reading',
                                                         '% Passing Math',
                                                         '% Overall Passing'
                                                         ].mean()

<ipython-input-78-7f02e64b1b64>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  scores_by_school_size = school_data_complete.groupby(['School Population','school_name'])['Avg. Reading Score',


In [79]:
scores_by_school_size


Avg. Reading Score  Avg. Math Score  \
School Population school_name                                                  
Large             Bailey High School              81.033963        77.048432   
                  Figueroa High School            81.158020        76.711767   
                  Ford High School                80.746258        77.102592   
                  Hernandez High School           80.934412        77.289752   
                  Huang High School               81.182722        76.629414   
                  Johnson High School             80.966394        77.072464   
                  Rodriguez High School           80.744686        76.842711   
                  Wilson High School              83.989488        83.274201   
Medium            Cabrera High School             83.975780        83.061895   
                  Griffin High School             83.816757        83.351499   
                  Shelton High School             83.725724        83.359455   
                  Thomas High School              83.848930        83.418349   
                  Wright High School              83.955000        83.682222   
Small             Holden High School              83.814988        83.803279   
                  Pena High School                84.044699        83.839917   

                                         % Passing Reading  % Passing Math  \
School Population school_name                                                
Large             Bailey High School             81.933280       66.680064   
                  Figueroa High School           80.739234       65.988471   
                  Ford High School               79.299014       68.309602   
                  Hernandez High School          80.862999       66.752967   
                  Huang High School              81.316421       65.683922   
                  Johnson High School            81.222432       66.057551   
                  Rodriguez High School          80.220055       66.366592   
                  Wilson High School             96.539641       93.867718   
Medium            Cabrera High School            97.039828       94.133477   
                  Griffin High School            97.138965       93.392371   
                  Shelton High School            95.854628       93.867121   
                  Thomas High School             97.308869       93.272171   
                  Wright High School             96.611111       93.333333   
Small             Holden High School             96.252927       92.505855   
                  Pena High School               95.945946       94.594595   

                                         % Overall Passing  
School Population school_name                               
Large             Bailey High School             74.306672  
                  Figueroa High School           73.363852  
                  Ford High School               73.804308  
                  Hernandez High School          73.807983  
                  Huang High School              73.500171  
                  Johnson High School            73.639992  
                  Rodriguez High School          73.293323  
                  Wilson High School             95.203679  
Medium            Cabrera High School            95.586652  
                  Griffin High School            95.265668  
                  Shelton High School            94.860875  
                  Thomas High School             95.290520  
                  Wright High School             94.972222  
Small             Holden High School             94.379391  
                  Pena High School               95.270270

In [80]:
# Create a copy of copy_school_sum and save as scores_by_school_type
scores_by_school_type = school_data_complete.copy()

# Convert scores_by_school_type to df
scores_by_school_type = pd.DataFrame(scores_by_school_type)

# Print scores_by_school_type cols to verify
# scores_by_school_type.columns

In [81]:
# Do a groupby on School Type
scores_by_school_type = school_data_complete.groupby(['type'])['Avg. Reading Score',
                                                         'Avg. Math Score',
                                                         '% Passing Reading',
                                                         '% Passing Math',
                                                         '% Overall Passing'
                                                         ].mean()

<ipython-input-81-574ee5cae1e0>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  scores_by_school_type = school_data_complete.groupby(['type'])['Avg. Reading Score',


In [82]:
scores_by_school_type.head()


,Avg. Reading Score,Avg. Math Score,% Passing Reading,% Passing Math,% Overall Passing
type,,,,,
Charter,83.902821,83.406183,96.645891,93.701821,95.173856
District,80.962485,76.987026,80.905249,66.518387,73.711818
